In [1]:
from utils import *

from time import time
from sklearn.model_selection import train_test_split

# Modelos
from sksurv.ensemble import RandomSurvivalForest

import warnings
warnings.filterwarnings("ignore")

## Parametros

In [2]:
table = 'Diabetes_avicena_survival.diabetes_final_3_annos'

variables_with_outliers = ['edad','IMC','HDL','LDL','trigliceridos','perimetro_abdominal']

variables_to_train = ['edad','IMC','HDL','LDL','trigliceridos','genero_paciente']

numeric_columns = ['edad','IMC','HDL','LDL','trigliceridos','perimetro_abdominal']
categoric_columns = ['genero_paciente','raza_paciente','nivel_academico_paciente','ant_cardio','med_hipertension','ant_familiar_dm','hace_ejercicio']
columns_not_in_count = ['ant_familiar_dm', 'raza_paciente','hace_ejercicio']
target = ['time_to_event','diabetes']

dict_var_categoricas = {
    # Nivel Academico
    "Ninguno" : 'ninguno',

    "Básica secundaria" : 'educacion_basica', 
    "Básica primaria" : 'educacion_basica',

    "Normalista" : 'educacion_media',
    "Bachillerato técnico" : 'educacion_media',
    "Técnica profesional" : 'educacion_media',
    "Tecnológica" : 'educacion_media',
    "Media académica o clásica" : 'educacion_media',

    "Profesional" : 'educacion_superior',
    "Especialización" : 'educacion_superior',
    "Preescolar" : 'educacion_superior',
    "Doctorado" : 'educacion_superior',
    "Maestría" : 'educacion_superior',
        
    # Ejercicio
    'Nunca' : 'No',
    '20 minutos' : '20 min',
    '40 minutos' : 'Mas de 20 min',
    '60 minutos' : 'Mas de 20 min',

    # Dicotomicas
    "1" : 'Si',
    '0' : 'No'
    

}

## Carga de datos

In [3]:
data = client_bq.query(f'SELECT * FROM {table}').result().to_dataframe()
print(f'Se trajo {data.shape} datos de pacientes')
data.head()

Se trajo (735003, 21) datos de pacientes


,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,...,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes,time_to_event
0,1003390652,2022,2,2022-02-01,26,105.233333,1.60,41.106770833,71.20,106.14,...,111.0,Femenino,Mestizo,Bachillerato técnico,0,0,0,None,0,34
1,1006578626,2022,2,2022-02-01,20,68.700000,1.77,21.928564589,56.25,143.00,...,NaN,Masculino,Otros,Profesional,1,0,0,None,0,36
2,1007218577,2022,2,2022-02-01,28,74.300000,1.78,23.450321929,45.10,96.64,...,NaN,Masculino,Otros,Ninguno,0,0,0,None,0,33
3,1010021506,2022,2,2022-02-01,36,71.000000,1.60,27.734375000,60.00,140.00,...,NaN,Masculino,Otros,Tecnológica,1,1,0,None,0,35
4,10241950,2022,2,2022-02-01,67,85.800000,1.72,29.002163332,41.00,82.00,...,105.0,Masculino,Otros,Básica secundaria,1,1,0,None,1,13


In [4]:
# Quitar outliers
data_clean = take_out_outliers(data, variables_to_train[:-1], verbose = False)
print(f'Hay {data_clean.shape[0]} datos de pacientes sin outliers')
data_clean.head()

Hay 660085 datos de pacientes sin outliers


,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,...,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes,time_to_event
1,1006578626,2022,2,2022-02-01,20.0,68.7,1.77,21.928565,56.25,143.00,...,NaN,Masculino,Otros,Profesional,1,0,0,None,0,36
2,1007218577,2022,2,2022-02-01,28.0,74.3,1.78,23.450322,45.10,96.64,...,NaN,Masculino,Otros,Ninguno,0,0,0,None,0,33
3,1010021506,2022,2,2022-02-01,36.0,71.0,1.60,27.734375,60.00,140.00,...,NaN,Masculino,Otros,Tecnológica,1,1,0,None,0,35
4,10241950,2022,2,2022-02-01,67.0,85.8,1.72,29.002163,41.00,82.00,...,105.0,Masculino,Otros,Básica secundaria,1,1,0,None,1,13
5,10529330,2022,2,2022-02-01,66.0,71.9,1.67,25.780774,46.50,46.30,...,NaN,Masculino,Otros,Básica secundaria,1,1,0,None,1,1


## Proceso

In [5]:
# Dataframe con los datos numericos
df_columns = data_clean[variables_to_train]
df_columns['IMC'] = np.round(df_columns['IMC'].astype(float),2)
df_columns.genero_paciente = df_columns.genero_paciente.replace({'Femenino' : 0, 'Masculino' : 1})

print(f'Hay {df_columns.shape[0]} pacientes para entrenar la linea base')
df_columns.head()

Hay 660085 pacientes para entrenar la linea base


,edad,IMC,HDL,LDL,trigliceridos,genero_paciente
1,20.0,21.93,56.25,143.00,100.05,1
2,28.0,23.45,45.10,96.64,64.68,1
3,36.0,27.73,60.00,140.00,117.00,1
4,67.0,29.00,41.00,82.00,203.00,1
5,66.0,25.78,46.50,46.30,109.00,1


In [6]:
df_train = escalar(df_columns)
df_train.columns = df_columns.columns
df_train.head()

,edad,IMC,HDL,LDL,trigliceridos,genero_paciente
0,0.024096,0.269439,0.644531,0.629771,0.312852,1.0
1,0.120482,0.332640,0.470313,0.408588,0.202251,1.0
2,0.216867,0.510603,0.703125,0.615458,0.365854,1.0
3,0.590361,0.563410,0.406250,0.338740,0.634772,1.0
4,0.578313,0.429522,0.492188,0.168416,0.340838,1.0


In [7]:
# Creacion variable target
target = data_clean[['diabetes','time_to_event']]
target['target'] = target.apply(lambda x: (bool(x.diabetes), x.time_to_event), axis = 1)
target = target['target']
target = np.array(target, dtype=[('event', np.bool_), ('time', np.int32)])

target[:5]

array([(False, 36), (False, 33), (False, 35), ( True, 13), ( True,  1)],
      dtype=[('event', '?'), ('time', '<i4')])

In [8]:
print('Data final para entrenar el modelo')
df_train.head()

print('Variable objetivo del modelo')
target[:5]

Data final para entrenar el modelo
Variable objetivo del modelo


array([(False, 36), (False, 33), (False, 35), ( True, 13), ( True,  1)],
      dtype=[('event', '?'), ('time', '<i4')])

In [55]:
## Entrenamiento del modelo

#tiempo inicial
t_1 = time()

y = target
X = df_train

# Conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=20)

print(f'Las dimensiones del entrenamiento son {X_train.shape} para X_train, {y_train.shape} para y_train')
print(f'Las dimensiones del entrenamiento son {X_test.shape} para X_test, {y_test.shape} para y_test')

# Definir el modelo (Random Forest)
rsf = RandomSurvivalForest(
    max_depth=100, min_samples_leaf=50, min_samples_split=30,
                     n_estimators=50, n_jobs=-1
)

# Entrenar el modelo
rsf.fit(X_train, y_train)

print('El modelo tiene un valor C-index de :',rsf.score(X_test, y_test))
print(f'Se demoro un total de {((time() - t_1)/60)} minutos')

Las dimensiones del entrenamiento son (495063, 6) para X_train, (495063,) para y_train
Las dimensiones del entrenamiento son (165022, 6) para X_test, (165022,) para y_test
El modelo tiene un valor C-index de : 0.566607368174622
Se demoro un total de 24.311487154165903 minutos
